In [7]:
import torch

In [ ]:
from _src import ModelConfig
from _src import GaussianProcess
from _src import BraninHoo 

In [8]:
from botorch.utils.constants import get_constants_like

from botorch.utils.probability.utils import (
    log_ndtr as log_Phi,
    log_phi,
    ndtr as Phi,
    phi,
)

In [13]:
import plotly.graph_objects as go

x = torch.linspace(-200, 500, 1000)
y = log_Phi(x)

fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=y))
fig.show()

In [17]:
import plotly.graph_objects as go

x = torch.linspace(-200, 500, 1000)
y = log_phi(x)

fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=y))
fig.show()

In [20]:
import plotly.graph_objects as go

x = torch.linspace(-10, 10, 1000)
y = phi(x)

fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=y))
fig.show()

In [22]:
arr = torch.linspace(-1,1,10).reshape(-1,1)
arr

tensor([[-1.0000],
        [-0.7778],
        [-0.5556],
        [-0.3333],
        [-0.1111],
        [ 0.1111],
        [ 0.3333],
        [ 0.5556],
        [ 0.7778],
        [ 1.0000]])

In [33]:
a = arr.squeeze(-2).squeeze(-1)
a

tensor([-1.0000, -0.7778, -0.5556, -0.3333, -0.1111,  0.1111,  0.3333,  0.5556,
         0.7778,  1.0000])

In [31]:
arr.clamp_min(0.5)

tensor([[0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5556],
        [0.7778],
        [1.0000]])

In [32]:
b = arr @ arr.T
b

tensor([[ 1.0000,  0.7778,  0.5556,  0.3333,  0.1111, -0.1111, -0.3333, -0.5556,
         -0.7778, -1.0000],
        [ 0.7778,  0.6049,  0.4321,  0.2593,  0.0864, -0.0864, -0.2593, -0.4321,
         -0.6049, -0.7778],
        [ 0.5556,  0.4321,  0.3086,  0.1852,  0.0617, -0.0617, -0.1852, -0.3086,
         -0.4321, -0.5556],
        [ 0.3333,  0.2593,  0.1852,  0.1111,  0.0370, -0.0370, -0.1111, -0.1852,
         -0.2593, -0.3333],
        [ 0.1111,  0.0864,  0.0617,  0.0370,  0.0123, -0.0123, -0.0370, -0.0617,
         -0.0864, -0.1111],
        [-0.1111, -0.0864, -0.0617, -0.0370, -0.0123,  0.0123,  0.0370,  0.0617,
          0.0864,  0.1111],
        [-0.3333, -0.2593, -0.1852, -0.1111, -0.0370,  0.0370,  0.1111,  0.1852,
          0.2593,  0.3333],
        [-0.5556, -0.4321, -0.3086, -0.1852, -0.0617,  0.0617,  0.1852,  0.3086,
          0.4321,  0.5556],
        [-0.7778, -0.6049, -0.4321, -0.2593, -0.0864,  0.0864,  0.2593,  0.4321,
          0.6049,  0.7778],
        [-1.0000, -

In [34]:
b.view(a.shape)

RuntimeError: shape '[10]' is invalid for input of size 100

In [36]:
from botorch.models import SingleTaskGP

In [39]:
X = torch.rand(10, 2)
y = torch.sin(X).sum(dim=-1, keepdim=True)
y

tensor([[0.3258],
        [1.3658],
        [0.9392],
        [1.2687],
        [0.5093],
        [0.9720],
        [0.1295],
        [1.2315],
        [0.8305],
        [1.1049]])

In [40]:
model = SingleTaskGP(X, y)
model

/var/folders/_b/b62kq0490fn2ptq7m4mbp8yr0000gp/T/ipykernel_62934/2198352611.py:1: InputDataWarning:

The model inputs are of type torch.float32. It is strongly recommended to use double precision in BoTorch, as this improves both precision and stability and can help avoid numerical errors. See https://github.com/pytorch/botorch/discussions/1444



SingleTaskGP(
  (likelihood): GaussianLikelihood(
    (noise_covar): HomoskedasticNoise(
      (noise_prior): LogNormalPrior()
      (raw_noise_constraint): GreaterThan(1.000E-04)
    )
  )
  (mean_module): ConstantMean()
  (covar_module): RBFKernel(
    (lengthscale_prior): LogNormalPrior()
    (raw_lengthscale_constraint): GreaterThan(2.500E-02)
  )
  (outcome_transform): Standardize()
)

In [44]:
model.posterior(X).variance

tensor([[0.0012],
        [0.0012],
        [0.0012],
        [0.0011],
        [0.0012],
        [0.0008],
        [0.0012],
        [0.0009],
        [0.0010],
        [0.0009]], grad_fn=<UnsqueezeBackward0>)

In [46]:
torch.tensor(3).clamp_min(1e-12).sqrt()

tensor(1.7321)

In [ ]:
from torch import Tensor

class AnalyticAcquisitionFucntion:
    def __init__(self, model, maximize):
        self.model = model
        self.maximize = maximize

    def __call__(self, X):
        X = X.to(self.device)
        mean, std = self._compute_mean_std(X)
        return self._compute_acquisition(mean, std)
    
    def _compute_mean_std(self, X):
        mean, std = self.model.predict(X, return_std=True)
        return mean, std

    def _compute_acquisition(self, mean, std):
        raise NotImplementedError
    
    
def _scaled_improvement(
    mean: Tensor, sigma: Tensor, best_f: Tensor, maximize: bool
) -> Tensor:
    """Returns `u = (mean - best_f) / sigma`, -u if maximize == True."""
    u = (mean - best_f) / sigma
    return u if maximize else -u
    



In [ ]:
class AnalyticAcquisitionFunction(AcquisitionFunction, ABC):
    """Base class for analytic acquisition functions."""

    def __init__(self, model: Model) -> None:
        r"""Base constructor for analytic acquisition functions.

        Args:
            model: A fitted single-outcome model.
        """
        super().__init__(model=model)
        if model.num_outputs != 1:
            raise UnsupportedError("Model must be single-output.")

    def _mean_and_sigma(
        self, X: Tensor, compute_sigma: bool = True, min_var: float = 1e-12
    ) -> tuple[Tensor, Tensor | None]:
        """Computes the first and second moments of the model posterior.

        Args:
            X: `batch_shape x q x d`-dim Tensor of model inputs.
            compute_sigma: Boolean indicating whether or not to compute the second
                moment (default: True).
            min_var: The minimum value the variance is clamped too. Should be positive.

        Returns:
            A tuple of tensors containing the first and second moments of the model
            posterior. Removes the last two dimensions if they have size one. Only
            returns a single tensor of means if compute_sigma is True.
        """
        self.to(device=X.device)  # ensures buffers / parameters are on the same device
        posterior = self.model.posterior(X=X)
        mean = posterior.mean.squeeze(-2).squeeze(-1)  # removing redundant dimensions
        if not compute_sigma:
            return mean, None
        sigma = posterior.variance.clamp_min(min_var).sqrt().view(mean.shape)
        return mean, sigma

In [ ]:
class LogProbabilityOfImprovement(AnalyticAcquisitionFunction):
    r"""Single-outcome Log Probability of Improvement.

    Logarithm of the probability of improvement over the current best observed value,
    computed using the analytic formula under a Normal posterior distribution. Only
    supports the case of q=1. Requires the posterior to be Gaussian. The model must be
    single-outcome.

    `logPI(x) = log(P(y >= best_f)), y ~ f(x)`

    Example:
        >>> model = SingleTaskGP(train_X, train_Y)
        >>> LogPI = LogProbabilityOfImprovement(model, best_f=0.2)
        >>> log_pi = LogPI(test_X)
    """

    _log: bool = True

    def __init__(self, model: Model, best_f: float | Tensor, maximize: bool = True):
        r"""Single-outcome Probability of Improvement.

        Args:
            model: A fitted single-outcome model.
            best_f: Either a scalar or a `b`-dim Tensor (batch mode) representing
                the best function value observed so far (assumed noiseless).
            maximize: If True, consider the problem a maximization problem.
        """
        super().__init__(model=model)
        self.register_buffer("best_f", torch.as_tensor(best_f))
        self.maximize = maximize

    @t_batch_mode_transform(expected_q=1)
    def forward(self, X: Tensor) -> Tensor:
        r"""Evaluate the Log Probability of Improvement on the candidate set X.

        Args:
            X: A `(b1 x ... bk) x 1 x d`-dim batched tensor of `d`-dim design points.

        Returns:
            A `(b1 x ... bk)`-dim tensor of Log Probability of Improvement values at
            the given design points `X`.
        """
        mean, sigma = self._mean_and_sigma(X)
        u = _scaled_improvement(mean, sigma, self.best_f, self.maximize)
        return log_Phi(u)


class ProbabilityOfImprovement(AnalyticAcquisitionFunction):
    r"""Single-outcome Probability of Improvement.

    Probability of improvement over the current best observed value, computed
    using the analytic formula under a Normal posterior distribution. Only
    supports the case of q=1. Requires the posterior to be Gaussian. The model
    must be single-outcome.

    `PI(x) = P(y >= best_f), y ~ f(x)`

    Example:
        >>> model = SingleTaskGP(train_X, train_Y)
        >>> PI = ProbabilityOfImprovement(model, best_f=0.2)
        >>> pi = PI(test_X)
    """

    def __init__(self, model: Model, best_f: float | Tensor, maximize: bool = True):
        r"""Single-outcome Probability of Improvement.

        Args:
            model: A fitted single-outcome model.
            best_f: Either a scalar or a `b`-dim Tensor (batch mode) representing
                the best function value observed so far (assumed noiseless).
            maximize: If True, consider the problem a maximization problem.
        """
        super().__init__(model=model)
        self.register_buffer("best_f", torch.as_tensor(best_f))
        self.maximize = maximize

    @t_batch_mode_transform(expected_q=1)
    def forward(self, X: Tensor) -> Tensor:
        r"""Evaluate the Probability of Improvement on the candidate set X.

        Args:
            X: A `(b1 x ... bk) x 1 x d`-dim batched tensor of `d`-dim design points.

        Returns:
            A `(b1 x ... bk)`-dim tensor of Probability of Improvement values at the
            given design points `X`.
        """
        mean, sigma = self._mean_and_sigma(X)
        u = _scaled_improvement(mean, sigma, self.best_f, self.maximize)
        return Phi(u)

In [ ]:
class ExpectedImprovement(AnalyticAcquisitionFunction):
    r"""Single-outcome Expected Improvement (analytic).

    Computes classic Expected Improvement over the current best observed value,
    using the analytic formula for a Normal posterior distribution.

    Example:
        >>> model = SingleTaskGP(train_X, train_Y)
        >>> EI = ExpectedImprovement(model, best_f=0.2)
        >>> ei = EI(test_X)
    """

    def __init__(self, model: Model, best_f: float | Tensor, maximize: bool = True):
        r"""Single-outcome Expected Improvement (analytic).

        Args:
            model: A fitted single-outcome model.
            best_f: Either a scalar or a `b`-dim Tensor (batch mode) representing
                the best function value observed so far (assumed noiseless).
            maximize: If True, consider the problem a maximization problem.
        """
        legacy_ei_numerics_warning(legacy_name=type(self).__name__)
        super().__init__(model=model)
        self.register_buffer("best_f", torch.as_tensor(best_f))
        self.maximize = maximize

    @t_batch_mode_transform(expected_q=1)
    def forward(self, X: Tensor) -> Tensor:
        r"""Evaluate Expected Improvement on the candidate set X.

        Args:
            X: A `(b1 x ... bk) x 1 x d`-dim batched tensor of `d`-dim design points.

        Returns:
            A `(b1 x ... bk)`-dim tensor of Expected Improvement values at the
            given design points `X`.
        """
        mean, sigma = self._mean_and_sigma(X)
        u = _scaled_improvement(mean, sigma, self.best_f, self.maximize)
        return sigma * _ei_helper(u)


class LogExpectedImprovement(AnalyticAcquisitionFunction):
    r"""Single-outcome Log Expected Improvement (analytic).

    Computes the logarithm of the classic Expected Improvement acquisition function,
    in a numerically robust manner.

    Example:
        >>> model = SingleTaskGP(train_X, train_Y)
        >>> LogEI = LogExpectedImprovement(model, best_f=0.2)
        >>> ei = LogEI(test_X)
    """

    _log: bool = True

    def __init__(self, model: Model, best_f: float | Tensor, maximize: bool = True):
        r"""Logarithm of single-outcome Expected Improvement (analytic).

        Args:
            model: A fitted single-outcome model.
            best_f: Either a scalar or a `b`-dim Tensor (batch mode) representing
                the best function value observed so far (assumed noiseless).
            maximize: If True, consider the problem a maximization problem.
        """
        super().__init__(model=model)
        self.register_buffer("best_f", torch.as_tensor(best_f))
        self.maximize = maximize

    @t_batch_mode_transform(expected_q=1)
    def forward(self, X: Tensor) -> Tensor:
        r"""Evaluate logarithm of Expected Improvement on the candidate set X.

        Args:
            X: A `(b1 x ... bk) x 1 x d`-dim batched tensor of `d`-dim design points.

        Returns:
            A `(b1 x ... bk)`-dim tensor of the logarithm of the Expected Improvement
            values at the given design points `X`.
        """
        mean, sigma = self._mean_and_sigma(X)
        u = _scaled_improvement(mean, sigma, self.best_f, self.maximize)
        return _log_ei_helper(u) + sigma.log()

In [ ]:






class UpperConfidenceBound(AnalyticAcquisitionFunction):
    r"""Single-outcome Upper Confidence Bound (UCB).

    Analytic upper confidence bound that comprises of the posterior mean plus an
    additional term: the posterior standard deviation weighted by a trade-off
    parameter, `beta`.

    Example:
        >>> model = SingleTaskGP(train_X, train_Y)
        >>> UCB = UpperConfidenceBound(model, beta=0.2)
        >>> ucb = UCB(test_X)
    """

    def __init__(self, model: Model, beta: float | Tensor, maximize: bool = True):
        r"""Single-outcome Upper Confidence Bound.

        Args:
            model: A fitted single-outcome GP model.
            beta: Either a scalar or a one-dim tensor with `b` elements (batch mode)
                representing the trade-off parameter between mean and covariance.
            maximize: If True, consider the problem a maximization problem.
        """
        super().__init__(model=model)
        self.register_buffer("beta", torch.as_tensor(beta))
        self.maximize = maximize

    @t_batch_mode_transform(expected_q=1)
    def forward(self, X: Tensor) -> Tensor:
        r"""Evaluate the Upper Confidence Bound on the candidate set X.

        Args:
            X: A `(b1 x ... bk) x 1 x d`-dim batched tensor of `d`-dim design points.

        Returns:
            A `(b1 x ... bk)`-dim tensor of Upper Confidence Bound values at the
            given design points `X`.
        """
        mean, sigma = self._mean_and_sigma(X)
        return (mean if self.maximize else -mean) + self.beta.sqrt() * sigma


# --------------- Helper functions for analytic acquisition functions. ---------------


def _scaled_improvement(
    mean: Tensor, sigma: Tensor, best_f: Tensor, maximize: bool
) -> Tensor:
    """Returns `u = (mean - best_f) / sigma`, -u if maximize == True."""
    u = (mean - best_f) / sigma
    return u if maximize else -u


def _ei_helper(u: Tensor) -> Tensor:
    """Computes phi(u) + u * Phi(u), where phi and Phi are the standard normal
    pdf and cdf, respectively. This is used to compute Expected Improvement.
    """
    return phi(u) + u * Phi(u)


def _log_ei_helper(u: Tensor) -> Tensor:

    # the following two numbers are needed for _log_ei_helper
    _neg_inv_sqrt2 = -(2**-0.5)
    _log_sqrt_pi_div_2 = math.log(math.pi / 2) / 2

    """Accurately computes log(phi(u) + u * Phi(u)) in a differentiable manner for u in
    [-10^100, 10^100] in double precision, and [-10^20, 10^20] in single precision.
    Beyond these intervals, a basic squaring of u can lead to floating point overflow.
    In contrast, the implementation in _ei_helper only yields usable gradients down to
    u ~ -10. As a consequence, _log_ei_helper improves the range of inputs for which a
    backward pass yields usable gradients by many orders of magnitude.
    """
    if not (u.dtype == torch.float32 or u.dtype == torch.float64):
        raise TypeError(
            f"LogExpectedImprovement only supports torch.float32 and torch.float64 "
            f"dtypes, but received {u.dtype = }."
        )
    # The function has two branching decisions. The first is u < bound, and in this
    # case, just taking the logarithm of the naive _ei_helper implementation works.
    bound = -1
    u_upper = u.masked_fill(u < bound, bound)  # mask u to avoid NaNs in gradients
    log_ei_upper = _ei_helper(u_upper).log()

    # When u <= bound, we need to be more careful and rearrange the EI formula as
    # log(phi(u)) + log(1 - exp(w)), where w = log(abs(u) * Phi(u) / phi(u)).
    # To this end, a second branch is necessary, depending on whether or not u is
    # smaller than approximately the negative inverse square root of the machine
    # precision. Below this point, numerical issues in computing log(1 - exp(w)) occur
    # as w approaches zero from below, even though the relative contribution to log_ei
    # vanishes in machine precision at that point.
    neg_inv_sqrt_eps = -1e6 if u.dtype == torch.float64 else -1e3

    # mask u for to avoid NaNs in gradients in first and second branch
    u_lower = u.masked_fill(u > bound, bound)
    u_eps = u_lower.masked_fill(u < neg_inv_sqrt_eps, neg_inv_sqrt_eps)
    # compute the logarithm of abs(u) * Phi(u) / phi(u) for moderately large negative u
    w = _log_abs_u_Phi_div_phi(u_eps)

    # 1) Now, we use a special implementation of log(1 - exp(w)) for moderately
    # large negative numbers, and
    # 2) capture the leading order of log(1 - exp(w)) for very large negative numbers.
    # The second special case is technically only required for single precision numbers
    # but does "the right thing" regardless.
    log_ei_lower = log_phi(u) + (
        torch.where(
            u > neg_inv_sqrt_eps,
            log1mexp(w),
            # The contribution of the next term relative to log_phi vanishes when
            # w_lower << eps but captures the leading order of the log1mexp term.
            -2 * u_lower.abs().log(),
        )
    )
    return torch.where(u > bound, log_ei_upper, log_ei_lower)


def _log_abs_u_Phi_div_phi(u: Tensor) -> Tensor:

    # the following two numbers are needed for _log_ei_helper
    _neg_inv_sqrt2 = -(2**-0.5)
    _log_sqrt_pi_div_2 = math.log(math.pi / 2) / 2

    """Computes log(abs(u) * Phi(u) / phi(u)), where phi and Phi are the normal pdf
    and cdf, respectively. The function is valid for u < 0.

    NOTE: In single precision arithmetic, the function becomes numerically unstable for
    u < -1e3. For this reason, a second branch in _log_ei_helper is necessary to handle
    this regime, where this function approaches -abs(u)^-2 asymptotically.

    The implementation is based on the following implementation of the logarithm of
    the scaled complementary error function (i.e. erfcx). Since we only require the
    positive branch for _log_ei_helper, _log_abs_u_Phi_div_phi does not have a branch,
    but is only valid for u < 0 (so that _neg_inv_sqrt2 * u > 0).

        def logerfcx(x: Tensor) -> Tensor:
            return torch.where(
                x < 0,
                torch.erfc(x.masked_fill(x > 0, 0)).log() + x**2,
                torch.special.erfcx(x.masked_fill(x < 0, 0)).log(),
        )

    Further, it is important for numerical accuracy to move u.abs() into the
    logarithm, rather than adding u.abs().log() to logerfcx. This is the reason
    for the rather complex name of this function: _log_abs_u_Phi_div_phi.
    """
    # get_constants_like allocates tensors with the appropriate dtype and device and
    # caches the result, which improves efficiency.
    a, b = get_constants_like(values=(_neg_inv_sqrt2, _log_sqrt_pi_div_2), ref=u)
    return torch.log(torch.special.erfcx(a * u) * u.abs()) + b


In [ ]:
obj_func = BraninHoo()
print(f'Search space: {obj_func.search_space}')

config = ModelConfig(ard_num_dims=obj_func.search_space.shape[1])
surrogate = GaussianProcess(config)
acquisition = Acquisition()

In [4]:
from _src import Hartmann6
func = Hartmann6()

In [6]:
func.search_space.shape[0]

2